<H1> Es necesario contar con GPU y un entorno de trabajo Python 3.7</H1>

<H3> Empezamos clonando el proyecto desde Github</H3>

In [1]:
# Clonar repositorio git
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git

Cloning into 'Real-Time-Voice-Cloning'...
remote: Enumerating objects: 2363, done.
remote: Total 2363 (delta 0), reused 0 (delta 0), pack-reused 2363
Receiving objects: 100% (2363/2363), 360.72 MiB | 57.19 MiB/s, done.
Resolving deltas: 100% (1294/1294), done.


<H3>Nos movemos al directorio en el que vamos a trabajar</H3>

In [1]:
cd /home/usuario/Machine Learning de Combate/Real-Time-Voice-Cloning/

/home/usuario/Machine Learning de Combate/Real-Time-Voice-Cloning


<H3>Instalamos las primeras dependencias</H3>

In [2]:
# Instalamos dependencias
!pip install -r requirements.txt
!apt-get install -y libportaudio2

  Using cached https://files.pythonhosted.org/packages/32/67/559ca8408431c37ad3a17e859c8c291ea82f092354074baef482b98ffb7b/tensorflow_gpu-1.14.0-cp37-cp37m-manylinux1_x86_64.whl
Processing /root/.cache/pip/wheels/d0/f8/d5/8e3af3ee957feb9b403a060ebe72f7561887fef9dea658326e/umap_learn-0.3.10-cp37-none-any.whl
Processing /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a/visdom-0.1.8.9-cp37-none-any.whl
Processing /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1/webrtcvad-2.0.10-cp37-cp37m-linux_x86_64.whl
Processing /root/.cache/pip/wheels/07/36/47/a9a4d151332cbdaec564500af9704a0ad862cf554dcf4bfda0/librosa-0.7.1-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/61/42/3e92d7aa64295483fbca20a86c89b34d0cb43cffaadaffe028793902d790/matplotlib-3.1.2-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9b/af/4fc72f9d38e43b092e91e5b8cb9956d25b2e3ff8c75aed95df5569e4734e/num

<H3>Instalamos gdown para descargar el modelo pre-entrenado desde Google Drive y unzip para descomprimir</H3>

In [3]:
!pip install gdown
!apt install unzip

Processing /root/.cache/pip/wheels/a7/9d/16/9e0bda9a327ff2cddaee8de48a27553fb1efce73133593d066/gdown-3.8.3-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/93/83/71a2ee6158bb9f39a90c0dea1637f81d5eef866e188e1971a1b1ab01a35a/filelock-3.0.12-py3-none-any.whl
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


<H3>Descargamos el dataset pre-entrenado y lo descomprimimos</H3>

In [5]:
# Descargar dataset
!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
!unzip pretrained.zip

Downloading...
From: https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
To: /home/usuario/Machine Learning de Combate/Real-Time-Voice-Cloning/pretrained.zip
384MB [00:06, 55.5MB/s] 
Archive:  pretrained.zip
   creating: encoder/saved_models/
  inflating: encoder/saved_models/pretrained.pt  
   creating: synthesizer/saved_models/
   creating: synthesizer/saved_models/logs-pretrained/
   creating: synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: vocoder/saved_models/
   creating: vocoder/saved_models/pretrained/
  inflating: vocoder/saved_models/pre

<H3>Instalamos el paquete google-colab. Google Colab es un servicio cloud gratuito que soporta el uso de GPU y se utiliza para desarrollar aplicaciones usando librerías como Keras, TensorFlow, PyTorch y OpenCV</H3>

In [2]:
import warnings
warnings.filterwarnings('ignore')

<H3>Instalamos PyTorch. Es un paquete de Python diseñado para realizar cálculos numéricos haciendo uso de la programación de tensores. Además permite su ejecución en GPU para acelerar los cálculos.
Normalmente PyTorch es usado tanto para sustituir numpy y procesar los cálculos en GPU como para la investigación y desarrollo en el campo del machine learning, centrado principalmente en el desarrollo de redes neuronales.</H3>

In [5]:
pip install torch

  Using cached https://files.pythonhosted.org/packages/f9/34/2107f342d4493b7107a600ee16005b2870b5a0a5a165bdf5c5e7168a16a6/torch-1.3.1-cp37-cp37m-manylinux1_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


<H3>Este proyecto utiliza SV2TTS, un sistema Text-To-Speech con un vocoder que trabaja en tiempo real. En el siguiente bloque preparamos el entorno</H3>

In [3]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)


Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt


<H3>Y ya podemos iniciar una demostración</H3>

In [ ]:
run -i demo_cli.py

Arguments:
    enc_model_fpath:   encoder/saved_models/pretrained.pt
    syn_model_dir:     synthesizer/saved_models/logs-pretrained
    voc_model_fpath:   vocoder/saved_models/pretrained/pretrained.pt
    low_mem:           False
    no_sound:          False

Ejecutando una prueba de configuración...

Found 1 GPUs available. Using GPU 0 (Tesla M60) of compute capability 5.2 with 8.5Gb total memory.

Preparing the encoder, the synthesizer and the vocoder...
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt
Testing your configuration with small inputs.
	Testing the encoder...
	Testing the synthesizer... (loading the model will output a lot of text)

Constructing model: Tacotron


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function 

Reference voice: enter an audio filepath of a voice to be cloned (mp3, wav, m4a, flac, ...):
 Liam_Neeson.wav


Loaded file succesfully
Created the embedding


Write a sentence (+-20 words) to be synthesized:
 My name is Liam Neeson and I want to welcome you to Global Artificial Intelligence 2019 Madrid. I'm gonna look for you, I'm gonna find you and I'm gonna kill you


Created the mel spectrogram
Synthesizing the waveform:
{| ████████████████ 114000/115200 | Batch Size: 12 | Gen Rate: 12.3kHz | }float64

Saved output as demo_output_00.wav


